In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

folder = 'H:\\My Drive\\PROJECTS\\PSI 2022-2025\\XRF fundamentals vs. MVA'

In [2]:
# ACTUAL VALUES
meta = pd.read_csv(folder+'\\data\\meta_both.csv')
#format
meta.drop(columns='Sample_Name',inplace=True)
fold_col = [x for x in meta.columns if 'Folds' in x]
fold_df = meta[['pkey']+fold_col]
meta.drop(columns=fold_col, inplace=True)

## FP

In [3]:
# ACTUAL VALUES
meta = pd.read_csv(folder+'\\data\\meta_both.csv')
#format
meta.drop(columns='Sample_Name',inplace=True)
fold_col = [x for x in meta.columns if 'Folds' in x]
fold_df = meta[['pkey']+fold_col]
meta.drop(columns=fold_col, inplace=True)
# get elements
element_list = list(meta.columns[1:])
element_list.remove('Hg')
# continue formatting
new = [x+'_actual' for x in meta.columns[1:]]
new.insert(0,'pkey')
meta.columns=new

# PREDICTED VALUES
fp = pd.read_csv(folder+'\\instrument_predictions.csv')
# format
fp = fp.groupby('pellet_name', as_index=False).mean()
to_drop = [x for x in fp.columns if '+/-' in x]
fp.drop(columns=to_drop, inplace=True)
fp.columns = [x.split(' ')[0] for x in fp.columns]
new = [x+'_pred' for x in fp.columns[1:]]
new.insert(0,'pkey')
fp.columns=new

# merge
fp_full = meta.merge(fp)
#format
cols = list(fp_full.columns[1:])
cols.sort()
cols.insert(0,'pkey')
fp_full = fp_full[cols]
fp_full.to_csv(folder+'\\instrument_pred_true.csv', index=False)

rmsep_list=[]
r2_list=[]
adj_r2_list=[]
n_list=[]

for element in element_list:
    
    outliers = list(fold_df[fold_df[f'{element}_Folds']==-1]['pkey'])
    
    p = f'{element}_pred'
    t = f'{element}_actual'
    temp = fp_full[(~fp_full[p].isna())&
                   (~fp_full[t].isna())&
                   (~fp_full['pkey'].isin(outliers))].copy()
    n = len(temp)
    n_list.append(n)
    
    # RMSEP
    rmsep = sqrt(mean_squared_error(temp[t], temp[p]))
    rmsep_list.append(rmsep)
    
    # R2
    r2 = r2_score(temp[t], temp[p])
    adj_r2 = 1 - (1-r2)*(len(temp) - 1) / (len(temp) - (temp.shape[1] - 1) - 1)
    r2_list.append(r2)
    adj_r2_list.append(adj_r2)
    
fp_results = pd.DataFrame({
    'element':element_list,
    'n_test':n_list,
    'rmsep':rmsep_list,
    'r2':r2_list,
    'adj_r2':adj_r2_list
})
fp_results.to_csv(folder+'\\FP_RMSEP_results.csv', index=False)

## MVA

In [4]:
# using PLS because best %RMSEP results
model = 'PLS'
# using both filters bc lowest errors
filt = 'both'

sens_df = pd.read_csv(folder+'\\sensitivities.csv')
sens = sens_df[sens_df['filter']==filt]['median'][0]

info = pd.read_csv(folder+'\\data\\dataset_summary.csv')
element_list = info.element.values

loq_list=[]
rmsep_list=[]
r2_list=[]
adj_r2_list=[]
n_list=[]

pt_full = meta[['pkey']] # to make cumulative pred/true

for element in element_list:
    
    # LOQ
    coeff = pd.read_csv(f'{folder}\\models\\{filt}\\{element}_{model}_coefs.csv')
    vector = pow(pow(coeff['coef'], 2).sum(),0.5)  #square root of sum of squares
    loq = 10 * vector * sens
    loq_list.append(loq)
    
    # adjusted RMSEP
    p = f'{element}_pred'
    t = f'{element}_actual'
    p_t = pd.read_csv(f'{folder}\\models\\{filt}\\{element}_{model}_test_pred_true.csv')
    p_t = p_t[p_t[p]>loq] # remove those below LOQ
    pt_full = pt_full.merge(p_t, how='left') # add to full table
    
    n_test = len(p_t)
    n_list.append(n_test)
    
    rmsep = sqrt(mean_squared_error(p_t[t], p_t[p]))
    rmsep_list.append(rmsep)
    
    # R2
    r2 = r2_score(p_t[t], p_t[p])
    adj_r2 = 1 - (1-r2)*(len(p_t) - 1) / (len(p_t) - (p_t.shape[1] - 1) - 1)
    r2_list.append(r2)
    adj_r2_list.append(adj_r2)
    
mva_results = pd.DataFrame({
    'element':element_list,
    'loq':loq_list,
    'n_test':n_list,
    'rmsep':rmsep_list,
    'r2':r2_list,
    'adj_r2':adj_r2_list
})
mva_results.to_csv(folder+'\\MVA_LOQ_RMSEP_results.csv', index=False)
pt_full.to_csv(folder+'\\MVA_test_pred_true.csv', index=False)

## summarize

In [5]:
# format to compare
cols=['MVA_'+x for x in mva_results.columns[1:]]
cols.insert(0,'element')
mva_results.columns=cols

cols=['FP_'+x for x in fp_results.columns[1:]]
cols.insert(0,'element')
fp_results.columns=cols

df = mva_results.merge(fp_results)

# contextualize errors
info = pd.read_csv(folder+'\\data\dataset_summary.csv')
if list(info.element)==list(df.element):
    df['MVA_%rmsep'] = df['MVA_rmsep']/info['median']
    df['FP_%rmsep'] = df['FP_rmsep']/info['median']
    
df.to_csv(folder+'\\combined_results.csv', index=False)